<a href="https://colab.research.google.com/github/wanname777/tensorflow/blob/master/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
# 加载数据
imdb,info=tfds.load("imdb_reviews",
          with_info=True,
          as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete894YDR/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete894YDR/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete894YDR/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
print(imdb)
print('*'*50)
print(info)
print('*'*50)
# 赋值
train_data,test_data=imdb['train'],imdb['test']
print(train_data)
training_sentences=[]
training_labels=[]
testing_sentences=[]
testing_labels=[]

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>, 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>, 'unsupervised': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>}
**************************************************
tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 5

In [ ]:
# 固定的迭代方式
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [ ]:
# 一些超参数
vocab_size=10000
embedding_dim=16
max_length=120
trunc_type='post'
ovv_tok="<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
print(tf.__version__)

2.2.0


In [ ]:
# 初始化
tokenizer=Tokenizer(num_words=vocab_size,oov_token=ovv_tok)
# fit
tokenizer.fit_on_texts(training_sentences)
# 获取索引
word_index=tokenizer.word_index
# 转成序列
sequences=tokenizer.texts_to_sequences(training_sentences)
# 填充
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)
# 测试集转序列
testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
# 测试集填充
testing_padded=pad_sequences(testing_sequences,maxlen=max_length)

In [ ]:
# 又一种索引
reverse_word_index=tokenizer.index_word
def decode_review(text):
  # 调用dict的get，获取单词，并加入空格
  # 写的真好
  return ' '.join([reverse_word_index.get(i,'?') for i in text])
print(decode_review(padded[0]))
print(training_sentences[0])

? ? b this was an absolutely terrible movie don't be <OOV> in by christopher walken or michael <OOV> both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie's ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the <OOV> rebels were making their cases for <OOV> maria <OOV> <OOV> appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining <OOV> like christopher <OOV> good name i could barely sit through it
b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pa

In [ ]:
model=tf.keras.Sequential([
  # Turns positive integers (indexes) into dense vectors of fixed size.
  tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
  # 更多层见course3——3.9
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(6,activation='relu'),
  tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs=10
model.fit(
    padded,
    training_labels_final,
    epochs=num_epochs,
    validation_data=(testing_padded,testing_labels_final)
)

Epoch 1/10
782/782 [==============================] - 5s 7ms/step - loss: 0.4873 - accuracy: 0.7455 - val_loss: 0.3509 - val_accuracy: 0.8440
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2477 - accuracy: 0.9050 - val_loss: 0.3638 - val_accuracy: 0.8394
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.1082 - accuracy: 0.9690 - val_loss: 0.4469 - val_accuracy: 0.8240
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0289 - accuracy: 0.9957 - val_loss: 0.5345 - val_accuracy: 0.8231
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0061 - accuracy: 0.9997 - val_loss: 0.6012 - val_accuracy: 0.8260
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.6554 - val_accuracy: 0.8254
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 9.3695e-04 - accuracy: 1.0000 - val_loss: 0.7038 - val_accuracy: 0.8248
Ep

In [ ]:
e=model.layers[0]
# 获得权重，这里指每个单词所构成的向量权重
weights=e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [ ]:
import io
out_v=io.open('vece.tsv','w',encoding='utf-8')
out_m=io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1,vocab_size):
  word=reverse_word_index[word_num]
  embeddings=weights[word_num]
  out_m.write(word+'\n')
  out_v.write('\t'.join([str(x) for x in embeddings])+'\n')
out_v.close()
out_m.close()